In [21]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import time
from gym import envs
from collections import deque




class QCar(nn.Module):
    def __init__(self):
        super(QCar, self).__init__()
        self.Lin1 = nn.Linear(4, 100)
        self.Lin2 = nn.Linear(100, 300)
        self.Lin3 = nn.Linear(300, 50)
        self.Lin4 = nn.Linear(50, 2)
    
    def forward(self, x):
        x.unsqueeze(0)
        x = x.cuda()
        x = self.Lin1(x)
        x = self.Lin2(x)
        x = self.Lin3(x)
        x = self.Lin4(x)
        return x
    

def ChooseButton(vals, eps):
    #print(vals)
    action = 0
    if random.uniform(0, 1) < eps:
        action = random.randint(0, 1)
    else:
        action = torch.argmax(vals)
    return action

def Qlearn(batch_size, gamma):
    batch = random.sample(Qlog, batch_size)
    for qstate, qaction, qrew, qdone, qnextstate in batch:
        target = qrew
        if not qdone:
            target = (qrew + gamma*(torch.max(AI(qnextstate)).item()))
            #print(target)
            
        qout = AI(qstate)
        qtarget = qout.data
        qtarget[qaction] = target
        OptimF1.zero_grad()
        loss = LossF1(qout, qtarget)
        loss.backward()
        OptimF1.step()
        

In [22]:
try:
    env

except NameError:
    #env = gym.make('MountainCar-v0')
    env = gym.make('CartPole-v1')
    
else:
    env.close()
    #env = gym.make('MountainCar-v0')
    env = gym.make('CartPole-v1')
    
#print(env.action_space)
epsilon = 1.00
epsilonmin = .05
epsilondecay = .9995
    
AI = QCar()
AI.cuda()
tframes = 1000
epochs = 300
done = False
info = []
Qlog = deque(maxlen=1000)
button = [0]
LossF1 = nn.MSELoss()
OptimF1 = optim.Adam(AI.parameters(), lr=.0001)
action = 0
batch_size = 5
gamma = .95
totalrew = 0
for e in range(epochs):
    frames = tframes
    env.reset()
    info, rew, done, _ = env.step(0)
    info = torch.Tensor(info, device='cuda')
    totalrew = 0
    while not done:
        info2 = info
        output = AI(info)
        button = ChooseButton(output, epsilon)
        #print(button)
        try:
            info, rew, done, _ = env.step(button)
        except:
            info, rew, done, _ = env.step(button.item())
        info = torch.Tensor(info, device='cuda')
        Qlog.append((info2, button, rew, done, info))
        totalrew += rew
        if len(Qlog) > batch_size:
            Qlearn(batch_size, gamma)
            if epsilon > epsilonmin:
                epsilon = epsilon * epsilondecay
        print('Epsilon is: ' + str(epsilon))
        env.render()
        frames = frames - 1
        if done == True:
            print("Total reward this epoch: " + str(totalrew))
        if frames == 0:
            done = True
            
env.close()
torch.save(AI, "./QCarTrained.backup.pth")
print("Saved AI")

Epsilon is: 1.0
Epsilon is: 1.0
Epsilon is: 1.0
Epsilon is: 1.0
Epsilon is: 1.0
Epsilon is: 0.9995
Epsilon is: 0.9990002500000001
Epsilon is: 0.9985007498750001
Epsilon is: 0.9980014995000627
Epsilon is: 0.9975024987503127
Epsilon is: 0.9970037475009376
Epsilon is: 0.9965052456271871
Epsilon is: 0.9960069930043736
Epsilon is: 0.9955089895078715
Epsilon is: 0.9950112350131176
Epsilon is: 0.9945137293956111
Epsilon is: 0.9940164725309134
Epsilon is: 0.993519464294648
Epsilon is: 0.9930227045625007
Epsilon is: 0.9925261932102195
Epsilon is: 0.9920299301136144
Epsilon is: 0.9915339151485576
Total reward this epoch: 22.0
Epsilon is: 0.9910381481909833
Epsilon is: 0.990542629116888
Epsilon is: 0.9900473578023296
Epsilon is: 0.9895523341234285
Epsilon is: 0.9890575579563669
Epsilon is: 0.9885630291773887
Epsilon is: 0.9880687476628001
Epsilon is: 0.9875747132889687
Epsilon is: 0.9870809259323243
Epsilon is: 0.9865873854693582
Epsilon is: 0.9860940917766235
Epsilon is: 0.9856010447307353
Epsil

Epsilon is: 0.8789456096400869
Epsilon is: 0.8785061368352669
Epsilon is: 0.8780668837668493
Epsilon is: 0.8776278503249659
Epsilon is: 0.8771890363998035
Epsilon is: 0.8767504418816037
Epsilon is: 0.876312066660663
Epsilon is: 0.8758739106273327
Epsilon is: 0.8754359736720191
Epsilon is: 0.8749982556851831
Total reward this epoch: 20.0
Epsilon is: 0.8745607565573406
Epsilon is: 0.8741234761790619
Epsilon is: 0.8736864144409725
Epsilon is: 0.873249571233752
Epsilon is: 0.8728129464481352
Epsilon is: 0.8723765399749112
Epsilon is: 0.8719403517049238
Epsilon is: 0.8715043815290714
Epsilon is: 0.8710686293383069
Epsilon is: 0.8706330950236377
Epsilon is: 0.870197778476126
Epsilon is: 0.869762679586888
Epsilon is: 0.8693277982470946
Epsilon is: 0.8688931343479711
Epsilon is: 0.8684586877807972
Epsilon is: 0.8680244584369068
Epsilon is: 0.8675904462076884
Total reward this epoch: 17.0
Epsilon is: 0.8671566509845846
Epsilon is: 0.8667230726590923
Epsilon is: 0.8662897111227629
Epsilon is: 0.

Epsilon is: 0.7713871459409963
Epsilon is: 0.7710014523680259
Epsilon is: 0.7706159516418419
Epsilon is: 0.770230643666021
Epsilon is: 0.7698455283441881
Epsilon is: 0.769460605580016
Epsilon is: 0.7690758752772261
Epsilon is: 0.7686913373395875
Total reward this epoch: 35.0
Epsilon is: 0.7683069916709178
Epsilon is: 0.7679228381750823
Epsilon is: 0.7675388767559949
Epsilon is: 0.7671551073176169
Epsilon is: 0.7667715297639581
Epsilon is: 0.7663881439990762
Epsilon is: 0.7660049499270767
Epsilon is: 0.7656219474521132
Epsilon is: 0.7652391364783871
Epsilon is: 0.764856516910148
Epsilon is: 0.764474088651693
Epsilon is: 0.7640918516073671
Epsilon is: 0.7637098056815635
Epsilon is: 0.7633279507787228
Epsilon is: 0.7629462868033335
Epsilon is: 0.7625648136599319
Epsilon is: 0.762183531253102
Epsilon is: 0.7618024394874755
Epsilon is: 0.7614215382677318
Epsilon is: 0.761040827498598
Epsilon is: 0.7606603070848488
Total reward this epoch: 21.0
Epsilon is: 0.7602799769313064
Epsilon is: 0.75

Epsilon is: 0.6790253741346843
Epsilon is: 0.678685861447617
Epsilon is: 0.6783465185168932
Epsilon is: 0.6780073452576348
Epsilon is: 0.6776683415850061
Epsilon is: 0.6773295074142136
Epsilon is: 0.6769908426605066
Epsilon is: 0.6766523472391763
Epsilon is: 0.6763140210655568
Epsilon is: 0.6759758640550241
Epsilon is: 0.6756378761229966
Total reward this epoch: 32.0
Epsilon is: 0.6753000571849351
Epsilon is: 0.6749624071563427
Epsilon is: 0.6746249259527646
Epsilon is: 0.6742876134897883
Epsilon is: 0.6739504696830434
Epsilon is: 0.673613494448202
Epsilon is: 0.6732766877009779
Epsilon is: 0.6729400493571275
Epsilon is: 0.672603579332449
Epsilon is: 0.6722672775427828
Epsilon is: 0.6719311439040114
Epsilon is: 0.6715951783320595
Epsilon is: 0.6712593807428935
Epsilon is: 0.6709237510525221
Epsilon is: 0.6705882891769959
Epsilon is: 0.6702529950324074
Epsilon is: 0.6699178685348912
Epsilon is: 0.6695829096006238
Total reward this epoch: 18.0
Epsilon is: 0.6692481181458235
Epsilon is: 0

Epsilon is: 0.5968263714298979
Epsilon is: 0.596527958244183
Epsilon is: 0.596229694265061
Epsilon is: 0.5959315794179285
Epsilon is: 0.5956336136282195
Epsilon is: 0.5953357968214055
Epsilon is: 0.5950381289229948
Epsilon is: 0.5947406098585333
Total reward this epoch: 10.0
Epsilon is: 0.594443239553604
Epsilon is: 0.5941460179338273
Epsilon is: 0.5938489449248604
Epsilon is: 0.593552020452398
Epsilon is: 0.5932552444421718
Epsilon is: 0.5929586168199508
Epsilon is: 0.5926621375115408
Epsilon is: 0.5923658064427851
Epsilon is: 0.5920696235395638
Epsilon is: 0.5917735887277941
Epsilon is: 0.5914777019334302
Epsilon is: 0.5911819630824635
Epsilon is: 0.5908863721009223
Total reward this epoch: 13.0
Epsilon is: 0.5905909289148719
Epsilon is: 0.5902956334504145
Epsilon is: 0.5900004856336893
Epsilon is: 0.5897054853908725
Epsilon is: 0.5894106326481772
Epsilon is: 0.5891159273318531
Epsilon is: 0.5888213693681873
Epsilon is: 0.5885269586835032
Epsilon is: 0.5882326952041615
Epsilon is: 0.

...

Epsilon is: 0.32537376263076373
Epsilon is: 0.32521107574944835
Epsilon is: 0.3250484702115736
Epsilon is: 0.32488594597646786
Epsilon is: 0.32472350300347963
Epsilon is: 0.3245611412519779
Epsilon is: 0.32439886068135193
Epsilon is: 0.32423666125101125
Epsilon is: 0.3240745429203858
Epsilon is: 0.3239125056489256
Total reward this epoch: 11.0
Epsilon is: 0.32375054939610115
Epsilon is: 0.32358867412140313
Epsilon is: 0.3234268797843424
Epsilon is: 0.3232651663444503
Epsilon is: 0.3231035337612781
Epsilon is: 0.3229419819943975
Epsilon is: 0.3227805110034003
Epsilon is: 0.32261912074789867
Epsilon is: 0.32245781118752476
Epsilon is: 0.32229658228193103
Epsilon is: 0.3221354339907901
Epsilon is: 0.3219743662737947
Total reward this epoch: 12.0
Epsilon is: 0.3218133790906578
Epsilon is: 0.3216524724011125
Epsilon is: 0.32149164616491194
Epsilon is: 0.3213309003418295
Epsilon is: 0.3211702348916586
Epsilon is: 0.32100964977421276
Epsilon is: 0.32084914494932565
Epsilon is: 0.3206887203768

Epsilon is: 0.2881393378963581
Epsilon is: 0.28799526822740995
Epsilon is: 0.28785127059329624
Total reward this epoch: 9.0
Epsilon is: 0.2877073449579996
Epsilon is: 0.2875634912855206
Epsilon is: 0.2874197095398779
Epsilon is: 0.287275999685108
Epsilon is: 0.28713236168526546
Epsilon is: 0.28698879550442286
Epsilon is: 0.28684530110667067
Total reward this epoch: 7.0
Epsilon is: 0.28670187845611733
Epsilon is: 0.2865585275168893
Epsilon is: 0.28641524825313086
Epsilon is: 0.2862720406290043
Epsilon is: 0.28612890460868984
Epsilon is: 0.2859858401563855
Epsilon is: 0.2858428472363073
Epsilon is: 0.28569992581268916
Epsilon is: 0.28555707584978285
Epsilon is: 0.28541429731185797
Epsilon is: 0.285271590163202
Total reward this epoch: 11.0
Epsilon is: 0.28512895436812047
Epsilon is: 0.28498638989093644
Epsilon is: 0.284843896695991
Epsilon is: 0.284701474747643
Epsilon is: 0.2845591240102692
Epsilon is: 0.2844168444482641
Epsilon is: 0.28427463602604
Epsilon is: 0.284132498708027
Total r

Epsilon is: 0.2552935131499509
Epsilon is: 0.25516586639337596
Epsilon is: 0.25503828346017926
Epsilon is: 0.25491076431844917
Epsilon is: 0.25478330893629
Epsilon is: 0.25465591728182185
Epsilon is: 0.25452858932318095
Epsilon is: 0.25440132502851937
Total reward this epoch: 9.0
Epsilon is: 0.2542741243660051
Epsilon is: 0.2541469873038221
Epsilon is: 0.25401991381017025
Epsilon is: 0.2538929038532652
Epsilon is: 0.25376595740133856
Epsilon is: 0.2536390744226379
Epsilon is: 0.25351225488542656
Epsilon is: 0.2533854987579839
Epsilon is: 0.2532588060086049
Epsilon is: 0.2531321766056006
Epsilon is: 0.2530056105172978
Epsilon is: 0.2528791077120392
Total reward this epoch: 12.0
Epsilon is: 0.2527526681581832
Epsilon is: 0.2526262918241041
Epsilon is: 0.25249997867819207
Epsilon is: 0.252373728688853
Epsilon is: 0.2522475418245086
Epsilon is: 0.25212141805359634
Epsilon is: 0.25199535734456957
Epsilon is: 0.2518693596658973
Epsilon is: 0.25174342498606433
Total reward this epoch: 9.0
Eps

Total reward this epoch: 11.0
Epsilon is: 0.2257398330446324
Epsilon is: 0.22562696312811012
Epsilon is: 0.22551414964654606
Epsilon is: 0.22540139257172281
Epsilon is: 0.22528869187543696
Epsilon is: 0.22517604752949924
Epsilon is: 0.2250634595057345
Epsilon is: 0.22495092777598164
Epsilon is: 0.22483845231209365
Total reward this epoch: 9.0
Epsilon is: 0.2247260330859376
Epsilon is: 0.22461367006939464
Epsilon is: 0.22450136323435996
Epsilon is: 0.2243891125527428
Epsilon is: 0.22427691799646643
Epsilon is: 0.22416477953746822
Epsilon is: 0.2240526971476995
Epsilon is: 0.22394067079912566
Epsilon is: 0.2238287004637261
Epsilon is: 0.22371678611349424
Total reward this epoch: 10.0
Epsilon is: 0.2236049277204375
Epsilon is: 0.2234931252565773
Epsilon is: 0.22338137869394903
Epsilon is: 0.22326968800460206
Epsilon is: 0.22315805316059978
Epsilon is: 0.22304647413401948
Epsilon is: 0.22293495089695248
Epsilon is: 0.22282348342150401
Epsilon is: 0.2227120716797933
Epsilon is: 0.2226007156

Epsilon is: 0.20010715936603957
Epsilon is: 0.20000710578635655
Total reward this epoch: 10.0
Epsilon is: 0.1999071022334634
Epsilon is: 0.19980714868234667
Epsilon is: 0.1997072451080055
Epsilon is: 0.1996073914854515
Epsilon is: 0.19950758778970878
Epsilon is: 0.19940783399581394
Epsilon is: 0.19930813007881604
Epsilon is: 0.19920847601377664
Total reward this epoch: 8.0
Epsilon is: 0.19910887177576977
Epsilon is: 0.1990093173398819
Epsilon is: 0.19890981268121197
Epsilon is: 0.19881035777487138
Epsilon is: 0.19871095259598395
Epsilon is: 0.19861159711968598
Epsilon is: 0.19851229132112616
Epsilon is: 0.1984130351754656
Epsilon is: 0.1983138286578779
Epsilon is: 0.19821467174354895
Total reward this epoch: 10.0
Epsilon is: 0.1981155644076772
Epsilon is: 0.19801650662547338
Epsilon is: 0.19791749837216066
Epsilon is: 0.1978185396229746
Epsilon is: 0.19771963035316312
Epsilon is: 0.19762077053798655
Epsilon is: 0.19752196015271756
Epsilon is: 0.1974231991726412
Epsilon is: 0.1973244875

In [ ]:
#Testing loop

try:
    tenv

except NameError:
    tenv = gym.make('CartPole-v1')
    
else:
    tenv.close()
    tenv = gym.make('CartPole-v1')
    
    
    
epsilon = 0
epsilonmin = 0
epsilondecay = 0
    
AI = QCar()
AI.cuda()
AI = torch.load("./QCarTrained.backup.pth")



tframes = 1000
epochs = 100
done = False
info = []
Qlog = deque(maxlen=1000)
button = [0]
LossF1 = nn.MSELoss()
OptimF1 = optim.Adam(AI.parameters(), lr=.001)
action = 0
batch_size = 5
gamma = .1
totalrew = 0
for e in range(epochs):
    frames = tframes
    env.reset()
    info, rew, done, _ = env.step(0)
    info = torch.Tensor(info, device='cuda')
    totalrew = 0
    while not done:
        output = AI(info)
        button = ChooseButton(output, epsilon)
        #print(button)
        try:
            info, rew, done, _ = env.step(button)
        except:
            info, rew, done, _ = env.step(button.item())
        info = torch.Tensor(info, device='cuda')
        totalrew += rew

        env.render()
        frames = frames - 1
        if done == True:
            print("Total reward this epoch: " + str(totalrew))
        if frames == 0:
            done = True
            
env.close()

Total reward this epoch: 9.0
Total reward this epoch: 9.0
Total reward this epoch: 9.0
Total reward this epoch: 8.0
Total reward this epoch: 9.0
Total reward this epoch: 8.0
Total reward this epoch: 8.0
Total reward this epoch: 8.0
Total reward this epoch: 9.0
Total reward this epoch: 7.0
Total reward this epoch: 8.0
Total reward this epoch: 9.0
Total reward this epoch: 8.0
Total reward this epoch: 9.0
Total reward this epoch: 9.0
Total reward this epoch: 8.0
Total reward this epoch: 8.0
Total reward this epoch: 10.0
Total reward this epoch: 9.0
Total reward this epoch: 9.0
Total reward this epoch: 7.0
Total reward this epoch: 7.0
Total reward this epoch: 8.0
Total reward this epoch: 8.0
Total reward this epoch: 7.0
Total reward this epoch: 7.0
Total reward this epoch: 9.0
Total reward this epoch: 8.0
Total reward this epoch: 7.0
Total reward this epoch: 9.0
Total reward this epoch: 8.0
Total reward this epoch: 8.0
Total reward this epoch: 8.0
Total reward this epoch: 7.0
Total reward 